In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import random
import sklearn
import tensorflow as tf

#Fonte:Nayak Sunita. Disponível em <https://www.learnopencv.com/deep-learning-based-object-detection-using-yolov3-with-opencv-python-c/>.
print(cv2.__version__)

confThreshold = 0.6  #Limite mínimo de confiança na identificação de um objeto
nmsThreshold = 0.4   #Non-maximum suppression threshold

#Carrega o nome das classes (carro,ônibus e caminhão) possíveis de serem identificados
classesFile = "YOLO\darknet-master\YOLOv3\coco.names";
classes = None
with open(classesFile, 'rt') as f:
    classes = f.read().rstrip('\n').split('\n')
    
                    
# Pega o nome das camadas de saída
def getOutputsNames(net):
    # Pega o nome de todas as camadas
    layersNames = net.getLayerNames()
    # Pega o nome das camadas de saída desconectadas
    return [layersNames[i[0] - 1] for i in net.getUnconnectedOutLayers()]


# Remove as bounding boxes com baixa confiança e non-maxima supressão
def postprocess(frame, outs):
    frameHeight = frame.shape[0]
    frameWidth = frame.shape[1]

    # Seleciona bounding boxes com confiança alta
    classIds = []
    confidences = []
    boxes = []
    efect_boxes = []
    efect_classIds = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            classId = np.argmax(scores)
            confidence = scores[classId]
            if confidence > confThreshold and (classId==1 or classId==2 or classId==3 or classId==5 or classId==7):
                center_x = int(detection[0] * frameWidth)
                center_y = int(detection[1] * frameHeight)
                width = int(detection[2] * frameWidth)
                height = int(detection[3] * frameHeight)
                left = int(center_x - width / 2)
                top = int(center_y - height / 2)
                #Se box for muito pequena considerar um carro
                if width < 200 and height<120 and (classId == 5 or classId == 7):
                    classId = 2
                classIds.append(classId)
                confidences.append(float(confidence))
                boxes.append([left, top, width, height])
 
    # Executa non maximum suppression para eliminar caixas que se sobrepoem com baixa confiança
    indices = cv2.dnn.NMSBoxes(boxes, confidences, confThreshold, nmsThreshold)
    id_centers = []
    for i in indices:
        i = i[0]
        box = boxes[i]
        efect_boxes.append(box)
        efect_classIds.append(classIds[i])
        left = box[0]
        top = box[1]
        width = box[2]
        height = box[3]
        id_centers.append([int(left+(width)/2),int(top+(height)/2)])
        drawPred(classIds[i], confidences[i], left, top, left + width, top + height,frame)
    return efect_boxes,efect_classIds,id_centers
        
# Desenha as bounding boxe
def drawPred(classId, conf, left, top, right, bottom,frame):
    
    cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255))
        
    label = '%.2f' % conf

    # Pega o nome da classe e sua confiança
    label = '%s:%s' % (classes[classId], label)

    # Mostre o tipo de objeto no topo da bounding box
    labelSize, baseLine = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
    top = max(top, labelSize[1])
    cv2.putText(frame, label, (left, top), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255))

C:\Users\lucas\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


3.4.4


In [2]:
import sys
from keras import backend as K
# Contador de eixos usado tanto para a linha central vermelha de contagem quanto 
# para a última abordagem de identificação e quantificação de eixos
class Axis_Counting:
    
    def __init__(self,axis_parameters_dist):
        self.ESRS = 0
        self.ESRD = 0
        self.ETD = 0
        self.ETT = 0
        
        self.objects = []
        self.num_fr_no_passing = 0
        self.axis_sequence = []
        self.axis_direction = []
        
        self.x1_counting_line_1 = int(width/2)

        self.boxes = []
        
        self.axis_parameters_dist = axis_parameters_dist 
        
    def box_scanned(self,num_fr,objects_set,trackers):
        def check_vehicle(x,y,objs,trackers):
            is_car = False
            box_tracker_index = 0 
            for i in range(len(objs[0])):
                x_min = objs[0][i][0]
                x_max = objs[0][i][0]+objs[0][i][2]
                y_min = objs[0][i][1]
                y_max = objs[0][i][1]+objs[0][i][3]
                
                for j,tracker in enumerate(trackers):
                    trk_x = tracker[1][0]
                    trk_y = tracker[1][1]
                    if trk_x>x_min and trk_x<x_max and trk_y>y_min and trk_y<y_max:
                        box_tracker_index = j

                if x>x_min and x<x_max and y>y_min and y<y_max:
                    if objs[1][i] == 2:
                        is_car = True
            return is_car,box_tracker_index
                
        box_line = []
        axis_passing = False
        for i,box in enumerate(self.boxes):
            x1 = box[0]
            x2 = box[1]
            center_x = box[2]
            center_y = box[3]
            is_car,box_tracker_index = check_vehicle(center_x,center_y,objects_set,trackers)
            if (x1<self.x1_counting_line_1-0 and x2<self.x1_counting_line_1-0) or (x1>self.x1_counting_line_1+0 and x2>self.x1_counting_line_1+0) or is_car:
                'Não faça nada'
            else:    
                axis_passing=True
                num_fr = 0
                box_line.append(box)
                self.axis_direction.append(center_x)
                break

        final_type = 0
        if axis_passing==False:
            num_fr+=1
        if num_fr >= 5:
            final_type = self.final_axis_type(self.axis_sequence)
            num_fr = 0
            self.axis_sequence.clear()  
            self.axis_direction.clear()
        return box_line,num_fr,final_type
    
   
    
    def final_axis_type(self,array):   
        final_type = 0
        if array!= None and len(array)>0:
            final_type = max(array)

        if final_type == 1:
            self.ESRD +=1
        elif final_type == 2:
            self.ETD +=1
        elif final_type >= 3:
            self.ETT +=1

        return final_type
        
    def axis_type(self,dists_rates):
        type_ = ''
        if dists_rates != None:
            num_wheel = 1
            for i,d in enumerate(dists_rates):
                #print(d[1])
                if d[0] < self.axis_parameters_dist:
                    num_wheel+=1
                elif d[1]<30:
                    num_wheel = 0
                    break
            if num_wheel>=3:
                type_ = 'ETT'
            elif num_wheel == 2:
                type_ = 'ETD'
            elif num_wheel == 1:
                type_ = 'ESRD'
            self.axis_sequence.append(num_wheel)
            
            #self.axis_sequence.append(type_)
            #print(type_)
        return type_
    
    def axis_distances(self,b):
        if len(b)!=0:
            box_center = b[0][2]
            box_width = abs(b[0][0]-b[0][1])
            distances = []
            individual_dist = []
            for i,box in enumerate(self.boxes):
                dist_1 = abs(box_center-box[2])
                if dist_1>10:
                    rate = box_width
                    distances.append([dist_1,rate])
            return distances

    def create_sess(self):
        modelWeights = "tensorflow\\models-master\\workspace\\training_demo\\trained-inference-graphs\\output_015\\frozen_inference_graph.pb"
        other = "tensorflow\\models-master\\workspace\\training_demo\\annotations\\label_map.pbtxt"
        net = cv2.dnn.readNetFromTensorflow(modelWeights,other)
        #net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
        #net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

        # Read the graph.
        with tf.gfile.GFile(modelWeights, 'rb') as f:
            graph_def = tf.GraphDef()
            graph_def.ParseFromString(f.read())

        sess = tf.Session()
        sess.graph.as_default()
        tf.import_graph_def(graph_def, name='')
        return sess

    def identify_axis(self,frame,objects_set,trackers,sess):
        axis_centers = []

        #inp = cv2.resize(frame, (350, 351))
        inp = frame[:, :, [2, 1, 0]]  # BGR2RGB

        # Run the model
        out = sess.run([sess.graph.get_tensor_by_name('num_detections:0'),
                        sess.graph.get_tensor_by_name('detection_scores:0'),
                        sess.graph.get_tensor_by_name('detection_boxes:0'),
                        sess.graph.get_tensor_by_name('detection_classes:0')],
                        feed_dict={'image_tensor:0': inp.reshape(1, inp.shape[0], inp.shape[1], 3)})

        total_score = out[1][0]
        total_bbox = out[2][0]
         
        # Visualize detected bounding boxes.
        num_detections = int(out[0][0])
        for i in range(num_detections):
            classId = int(out[3][0][i])
            score = float(out[1][0][i])
            bbox = [float(v) for v in out[2][0][i]]
            
            if score > 0.4:
                x = bbox[1] * width
                y = bbox[0] * height
                right = bbox[3] * width
                bottom = bbox[2] * height
                center_x = (x+right)/2
                center_y = (y+bottom)/2
                cv2.rectangle(frame, (int(x), int(y)), (int(right), int(bottom)), (125, 255, 51), thickness=2)
                self.boxes.append([x,right,center_x,center_y])
                axis_centers.append([center_x,center_y])
        
        box_line,num_fr,final_type = self.box_scanned(self.num_fr_no_passing,objects_set,trackers)
        self.num_fr_no_passing = num_fr
        self.axis_type(self.axis_distances(box_line))
        #print(len(boxes))
        self.boxes.clear()
            
        return frame, axis_centers,final_type


Using TensorFlow backend.


In [3]:
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(30,30))
# Encontra movimento na imagem
def find_movement(frame,subtractor):
    fgmask = subtractor.apply(frame)    
    fgmask = cv2.morphologyEx(fgmask, cv2.MORPH_DILATE, kernel)
    fgmask = cv2.morphologyEx(fgmask, cv2.MORPH_OPEN, kernel)
    (im2, contours, hierarchy) = cv2.findContours(fgmask, cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    
    #Matriz com as boxes de objetos em movimento
    movement_boxes = []
    
    #looping for contours
    for c in contours:
        if cv2.contourArea(c) < 5000:
            continue
        #get bounding box from countour
        (x, y, w, h) = cv2.boundingRect(c)
        movement_boxes.append([x,y,w,h])
    
    return movement_boxes,fgmask,contours

def draw_movement(frame, movements):
    #draw bounding box
    for move in movements:
        cv2.rectangle(frame, (move[0], move[1]), (move[0]+move[2], move[1]+move[3]), (0, 255, 0), 2) 

In [4]:
import cv2
import tkinter as tk

#Video a ser analisado
cap = cv2.VideoCapture('videos/videos_00/v_00_05.mp4')
#Definição do frame de início
start_frame = 0#26730,26670
cap.set(1,start_frame)
#Dimensões da imagem (largura e altura)
width = int(cap.get(3))
height = int(cap.get(4))

# Comandos para inicializar o video e realizar a marcação entre duas rodas de um eixo tandem
class Settings:
    def __init__(self,start_frame):
        self.running = 0
        self.points = []
        self.last_frame = 0
        self.axis_parameter_dist = 0
        self.start_frame = start_frame
        self.band = True
    def stop(self):
        def mouse_callback(event, x, y, flags, param):
                if event == cv2.EVENT_FLAG_LBUTTON:
                    if self.running == 0:
                        self.running = 1
                    else:
                        if len(self.points) == 0 or len(self.points) == 1:
                            self.points.append([x,y])
                if event == cv2.EVENT_FLAG_RBUTTON:
                    self.points.clear()
                    self.running = 0
        cv2.setMouseCallback('frame', mouse_callback)
        return self.running
    
    def draw(self,frame):
        if len(self.points)==2:
            img = cv2.line(frame,(self.points[0][0],self.points[0][1]),(self.points[1][0],self.points[1][1]),(0,0,255),2)
            
    def run(self,cap):
        cap.set(1,self.start_frame)
        while(cap.isOpened()):
            if self.band == True:
                if self.stop() == 0:
                    # Read a new frame
                    ok, frame = cap.read()
                    if not ok:
                        break
                    self.draw(frame)
                    cv2.imshow('frame',frame)

                elif self.stop() == 1 and len(self.points) == 2:
                    img = cv2.line(frame,(self.points[0][0],self.points[0][1]),(self.points[1][0],self.points[1][1]),(0,0,255),2)
                    cv2.imshow('frame',frame)
                    if cv2.waitKey() == ord('f'):
                        self.axis_parameter_dist_front = abs(self.points[0][0] - self.points[1][0]) 
                        self.band = False
                        self.stop == 0
                
            elif self.band == False:
                if self.stop() == 0:
                    # Read a new frame
                    ok, frame = cap.read()
                    if not ok:
                        break
                    self.draw(frame)
                    cv2.imshow('frame',frame)

                elif self.stop() == 1 and len(self.points) == 2:
                    img = cv2.line(frame,(self.points[0][0],self.points[0][1]),(self.points[1][0],self.points[1][1]),(0,0,255),2)
                    cv2.imshow('frame',frame)
                    if cv2.waitKey() == ord('f'):
                        self.axis_parameter_dist_back = abs(self.points[0][0] - self.points[1][0]) 
                        break
            if cv2.waitKey(1) & 0xFF == ord('r'):
                break
        cap.release()
        cv2.destroyAllWindows()
        print(self.axis_parameter_dist_front)
        print(self.axis_parameter_dist_back)
        return self.axis_parameter_dist_front,self.axis_parameter_dist_back

In [5]:
from time import time
from sklearn.cluster import DBSCAN

class run_video:
    def __init__(self):
        
        #Video a ser analisado
        self.cap = cv2.VideoCapture('videos/videos_00/v_00_05.mp4')
        cap_copy = cv2.VideoCapture('videos/videos_00/v_00_05.mp4')
        #Definição do frame de início
        self.start_frame = 0
        self.cap.set(1,self.start_frame)
        #Dimensões da imagem (largura e altura)
        width = int(self.cap.get(3))
        height = int(self.cap.get(4))
        
        #Quantidade de frames de todo o video
        self.total_num_frames = int(self.cap.get(7))
        #Número do frame para ser mostrado ao longo do video
        self.num_frame = 0
        #Intervalo de frames em que será avaliada a presença de veículos. Usado para aumentar a velocidade de processamento.
        self.rate_frame = 1

        #contagem dos veículos no sentido --->
        self.counting_left_right = [0,0,0,0]
        #contagem dos veículos no sentido <---
        self.counting_right_left = [0,0,0,0]
        #contagem dos eixos --->
        self.axis_left_right = [0,0,0,0]
        #contagem dos eixos <---
        self.axis_right_left = [0,0,0,0]

        #Posição da primeira linha de contagem para veículos de passeio (Carros)
        self.x1_count_line_light = int(width/2-25)
        self.x2_count_line_light = self.x1_count_line_light
        self.y1_count_line_light = int(height-120)
        self.y2_count_line_light = int(70)
        #Posição da segunda linha de contagem para veículos de passeio (Carros)
        self.x1_count_line2_light = int(width/2+25)
        self.x2_count_line2_light = self.x1_count_line2_light
        self.y1_count_line2_light = int(height-120)
        self.y2_count_line2_light = int(70)

        #Posição da primeira linha de contagem para veículos comerciais (Caminhões e Ônibus)
        self.x1_count_line_heavy = int(width/2-100)
        self.x2_count_line_heavy = self.x1_count_line_heavy
        self.y1_count_line_heavy = int(height-120)
        self.y2_count_line_heavy = int(70)

        #Posição da segunda linha de contagem para veículos comerciais (Caminhões e Ônibus)
        self.x1_count_line2_heavy = int(width/2+100)
        self.x2_count_line2_heavy = self.x1_count_line2_heavy
        self.y1_count_line2_heavy = int(height-120)
        self.y2_count_line2_heavy = int(70)
        
        self.seq_objs_count_left_right = []
        self.seq_objs_count_right_left = []
        
        self.bitrem_frame_count_left_right = 0
        self.bitrem_frame_alert_left_right = False
        self.bitrem_frame_count_right_left = 0
        self.bitrem_frame_alert_right_left = False

        # Inicialização dos parâmetros de processamento e identificação de objetos
        self.inpWidth = 224       #Largura da imagem a ser passada para a rede neural
        self.inpHeight = 224      #Altura da imagem a ser passada para a rede neural

        #Subtractor para encontrar movimento no frame
        self.subtractor = cv2.bgsegm.createBackgroundSubtractorMOG()

        #Matriz com objetos identificados na imagem
        self.objects = []

        #Arquivos de configuração para carreagmento da rede neural e de pesos a serem utilizados.
        modelConfiguration = "YOLO\darknet-master\YOLOv3\yolov3.cfg"
        modelWeights = "YOLO\darknet-master\YOLOv3\yolov3.weights"

        #Configura rede neural responsável por identificar veículos na imagem
        self.net = cv2.dnn.readNetFromDarknet(modelConfiguration, modelWeights)
        self.net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
        self.net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

        # Define the codec and create VideoWriter object
        fourcc = cv2.VideoWriter_fourcc(*'X264')
        self.out = cv2.VideoWriter('output.mp4',fourcc, 20.0, (width,height))
        
        settings = Settings(self.start_frame)
        self.axis_counting_dist,self.axis_counting_dist_2 = settings.run(cap_copy)
        self.axis_counting = Axis_Counting(self.axis_counting_dist)
        self.sess = self.axis_counting.create_sess()
        self.axis_sequence = []
        
        # Cálculo do tamanho dos tipos de caminhões (faixas A - 0m até 15m, B - 15m até 20m e C - >20m)
        # Utiliza-se o tamanho entre os eixos para calcular quantos pixels representa para as faixas A,B e C dos tamanhos e para cada faixa da rodovia (a mais próxima e mais afastada)
        # Distância em metros considerada entre duas rodas em um eixo tandem
        dist_rodas_tandem = 1.8
        
        width_metros = (width*dist_rodas_tandem)/self.axis_counting_dist
        print(width_metros)
        taxa_pixels_metro = width/width_metros
        
        faixa_a_metros = 15
        faixa_b_metros = 20
        faixa_c_metros = width_metros
        
        self.faixa_a_pixels = faixa_a_metros*taxa_pixels_metro
        self.faixa_b_pixels = faixa_b_metros*taxa_pixels_metro
        self.faixa_c_pixels = faixa_c_metros*taxa_pixels_metro
        
        self.id_centers = []
        self.count_id = 0
        
        self.run()
  
    def check_trackers(self,boxes):
        k = 0
        for o,obj in enumerate(self.objects):
            for box in boxes:
                if obj[1][0]+obj[1][2]/2<box[0] or obj[1][0]+obj[1][2]/2>box[0]+box[2] or obj[1][1]+obj[1][3]/2<box[1] or obj[1][1]+obj[1][3]/2>box[3]:
                    k = k + 1
            if k>=len(boxes):
                obj[13] = obj[13] + 1
            if obj[13] >= 100:
                self.objects.pop(o)

    def create_object(self,box,classIds,i,direct,frame,axis_pos,axis_quant,axis_type,sizes):
        bitrem_soma = 0
        if direct == True:
            if classIds[i] == 7:
                init_box = [box[0]+int(box[2]/3),box[1]+int(box[3]/2),30,30]
            else: 
                init_box = [box[0],box[1],box[2],box[3]]
        elif direct == False:
            if classIds[i] == 7:
                init_box = [box[0]+int(2*box[2]/3),box[1]+int(box[3]/2),30,30]
            else: 
                init_box = [box[0],box[1],box[2],box[3]]
        else:
            init_box = [box[0]+int(box[2]/2),box[1]+int(box[3]/2),30,30]
        
        #init_box = [box[0],box[1],box[2],box[3]]
        index = len(self.objects)
        tracker = cv2.TrackerMedianFlow_create()
        #tracker = cv2.TrackerCSRT_create()
        obj_no_box_count = 0
        success = tracker.init(frame,tuple(init_box))
        if direct == True:
            self.objects.append([tracker,tuple(init_box),classIds[i],False,axis_pos,axis_quant,axis_type,box,index,direct,sizes,bitrem_soma,self.bitrem_frame_alert_left_right,obj_no_box_count])
        elif direct == False:
            self.objects.append([tracker,tuple(init_box),classIds[i],False,axis_pos,axis_quant,axis_type,box,index,direct,sizes,bitrem_soma,self.bitrem_frame_alert_right_left,obj_no_box_count])
        #print('tracker added')
     
    # Initialize MultiTracker
    def tracker_config(self,boxes,objects_updated,classIds,frame,axis_centers,move_boxes):
        for i, box in enumerate(boxes):
            #determine the direction
            p1 = abs(self.x1_count_line_light-box[0])
            p2 = abs(box[0]+box[2]-self.x1_count_line_light)

            direction = True
            #if p1>p2 ---> True, if p1<p2 <--- False
            if p1>p2:
                direction = True
            elif p2>p1:
                direction = False
            p = p2/(p1+p2)

            k = 0
            for j, obj in enumerate(objects_updated):
                for move in move_boxes:
                    if box[0]>move[0] and box[0]+box[2]<move[0]+move[2] and box[1]>move[1] and box[1]+box[3]<move[1]+move[3]:
                        if classIds[i] == 7:
                            box = move
                            break
                if (obj[1][0]+obj[1][2]/2)<box[0] or (obj[1][0]+obj[1][2]/2)>box[0]+box[2] or (obj[1][1]+obj[1][3]/2)<box[1] or (obj[1][1]+obj[1][3]/2)>box[1]+box[3]: 
                    k = k+1
                else: 
                    # Matriz para adição dos eixos encontrados dentro de cada objeto
                    axis_inside_box = []
                    # Adicionar eixos dentro do objeto na matriz
                    for l, axis in enumerate(axis_centers):  
                        if axis[0]>box[0] and axis[0]<box[0]+box[2] and axis[1]>box[1] and axis[1]<box[1]+box[3]: 
                            axis_inside_box.append(axis[0])

                    # Ordena em ordem crescente os eixos em relação ao valores do eixo X
                    sort = sorted(axis_inside_box)
                    obj[4].append(sort)

                    # Matriz para adição dos tipos de espaçamento entre rodas encontrados (0 para distancias maiores que 2,4m e 1 para menores)        
                    axis_span_array = []
                    for v,s in enumerate(sort):

                        pos_val = 0
                        dist = 0

                        if v != 0:
                            dist = abs(sort[v]-sort[v-1])
                            if dist<self.axis_counting_dist*1.5:
                                pos_val = (sort[v]+sort[v-1])/2

                                if obj[9] == True:
                                    pos_per = abs(pos_val-(box[0]+box[2]))/box[2]

                                elif obj[9] == False:
                                    pos_per = abs(pos_val-box[0])/box[2]

                                axis_span_array.append([1,pos_per])

                            elif dist>=self.axis_counting_dist*1.5 and dist<self.axis_counting_dist*2.0:
                                pos_val = sort[v]

                                if obj[9] == True:
                                    pos_per = abs(pos_val-(box[0]+box[2]))/box[2]
                                elif obj[9] == False:
                                    pos_per = abs(pos_val-box[0])/box[2]
                                axis_span_array.append([2,pos_per])

                            elif dist>=self.axis_counting_dist*2.0:
                                pos_val = sort[v]

                                if obj[9] == True:
                                    pos_per = abs(pos_val-(box[0]+box[2]))/box[2]
                                elif obj[9] == False:
                                    pos_per = abs(pos_val-box[0])/box[2]
                                axis_span_array.append([0,pos_per])

                        elif v==0:
                            pos_val = sort[v]

                            if obj[9] == True:
                                pos_per = abs(pos_val-(box[0]+box[2]))/box[2]
                            elif obj[9] == False:
                                pos_per = abs(pos_val-box[0])/box[2]
                            axis_span_array.append([0,pos_per])


                    # Matriz para adição dos tipos de eixos
                    real_axis_type = []
                    # Variável auxiliar para cálculo do tipo de eixo
                    soma_tandem = 0
                    pos_aux = 0
                    soma_esrd = 0
                    last = -1
        
                    axis_span_array = sorted(axis_span_array)
                   
                    
                    for a,span in enumerate(axis_span_array):
                        if span[0] == 0 and soma_tandem != 0:
                            real_axis_type.append([soma_tandem,pos_aux])
                            soma_tandem = 0
                            pos_aux = 0
                        elif span[0] == 1 and a<len(axis_span_array)-1:
                            if soma_tandem == 0:
                                soma_tandem = soma_tandem+1
                                pos_aux = span[1]
                            elif soma_tandem != 0:
                                soma_tandem = soma_tandem+1
                                pos_aux = (pos_aux+span[1])/2
                        elif span[0] == 1 and a==len(axis_span_array)-1:
                            if soma_tandem == 0:
                                soma_tandem = soma_tandem+1
                                pos_aux = span[1]
                            elif soma_tandem != 0:
                                soma_tandem = soma_tandem+1
                                pos_aux = (pos_aux+span[1])/2
                            real_axis_type.append([soma_tandem,pos_aux])


                        if span[0] == 2:
                            if a == 0:
                                pos_aux = span[1]
                                real_axis_type.append([0,pos_aux])

                            if a==len(axis_span_array)-1:
                                pos_aux = (axis_span_array[a-1][1] + span[1])/2
                                real_axis_type.append([2,pos_aux])
                            else:
                                if (axis_span_array[a+1][0] == 0 and last == 0):
                                    pos_aux = (axis_span_array[a-1][1] + span[1])/2
                                    real_axis_type.append([2,pos_aux])
                            last = 2  

                        if span[0] == 0:
                            if a == 0 and span[1]<0.3:
                                pos_aux = span[1]
                                real_axis_type.append([0,pos_aux])
                            if a==len(axis_span_array)-1 and span[1]>0.7:
                                pos_aux = span[1]
                                real_axis_type.append([0,pos_aux])
                            elif a!=0 and a!=len(axis_span_array)-1:
                                if (axis_span_array[a+1][0] == 0 and last == 1) or (last == 0 and axis_span_array[a+1][0] == 0):
                                    pos_aux = span[1]
                                    real_axis_type.append([0,pos_aux])
                            last = 0
                        if span[0] == 1:
                            last = 1                            
                    # Atualiza valores relacionados aos eixos do objeto
                    obj[5].append(len(axis_inside_box))
                    obj[6].append(real_axis_type)
                    print(real_axis_type)
                    obj[7] = box
                    obj[10].append(box[2])

            if classIds[i] == 7 or classIds[i] == 5:
                x1 = self.x1_count_line_heavy
                x2 = self.x1_count_line2_heavy
                self.crossing_line(x1,x2,objects_updated,box,classIds,i,direction,k,frame,move_boxes)
            else:
                x1 = self.x1_count_line_light
                x2 = self.x1_count_line2_light
                self.crossing_line(x1,x2,objects_updated,box,classIds,i,direction,k,frame,move_boxes)

    def wheel_ident_avaliator(self,real_axis_set):
        intermediate_total = []
        final_total = []
        for axis_frame in real_axis_set:
            intermediate = []
            final = []
            for a in axis_frame:
                type_axis= a[0]
                pos_axis= a[1] 
                if pos_axis<=0.3:
                    print('RODA INICIAL DETECTADA')
                elif pos_axis>0.3 and pos_axis<=0.6:
                    #print('EIXO INTERMEDIÁRIO DETECTADO')
                    intermediate.append(type_axis)
                elif pos_axis>0.6:
                    #print('EIXO FINAL DETECTADO')  
                    final.append(type_axis)
            intermediate_total.append(intermediate)
            final_total.append(final)
        bigger_interm = 0
        bigger_index_interm = 0    
        for i,inter in enumerate(intermediate_total):
            if sum(inter)>bigger_interm:
                bigger_interm = sum(inter)
                bigger_index_interm = i
                
        bigger_final = 0        
        bigger_index_final = 0    
        for i,inter in enumerate(final_total):
            if sum(inter)>bigger_final:
                bigger_final = sum(inter)
                bigger_index_final = i
        print('INTERMEDIATE')
        print(intermediate_total[bigger_index_interm])
        print('FINAL')
        print(final_total[bigger_index_final])    
        return intermediate_total[bigger_index_interm],final_total[bigger_index_final]
    
    
    def crossing_line(self,x1_line,x2_line,objects_updated,box,classIds,i,direction,k,frame,move_boxes):     
        if box[0]<x1_line and box[0]+box[2]>x2_line:
            for j, obj in enumerate(objects_updated):
                if obj[9] == True:
                    if width-(obj[1][0]+obj[1][2]/2)<int(width/6):
                        self.objects.pop(j)
                elif obj[9] == False:
                    if (obj[1][0]+obj[1][2]/2)<int(width/6):
                        self.objects.pop(j)
            if k >= len(objects_updated):
                axis_pos = []
                axis_quant = []
                axis_type = []
                sizes = []
                self.create_object(box,classIds,i,direction,frame,axis_pos,axis_quant,axis_type,sizes)
                self.update_indexes()
    
    def update_indexes(self):
        if len(self.objects)>0:
            # Atualiza os índices dos objetos
            for i in self.objects:
                i[8] = self.objects.index(i)
            
    #Avalia se informações de eixos fazem sentido para cada parte do caminhão e contabiliza os eixos
    def truck_size(self,direction,bigger_size,axis_intermediate,axis_final):
        def makes_sense(axis_intermediate,axis_final):
            intermediate_sense = True
            final_sense = True
            if bigger_size<=self.faixa_a_pixels:
                unique_val = []
                for i in axis_intermediate:
                    for u in unique_val:
                        if u == i:
                            break
                        else:
                            unique_val.append(i)
                if len(axis_intermediate)>1:
                    intermediate_sense = False
                if len(unique_val)>0:    
                    if max(unique_val)>1:
                        intermediate_sense = False  
                if len(axis_final)>1:
                    final_sense = False
            if bigger_size>self.faixa_a_pixels and bigger_size<=self.faixa_b_pixels:
                if len(axis_intermediate)!=1:
                    intermediate_sense = False 
            if bigger_size>self.faixa_b_pixels:
                if len(axis_intermediate)<1:
                    intermediate_sense = False  
                if len(axis_final)<1:
                    final_sense = False
            return intermediate_sense, final_sense
            
        def counting(intermediate_sense,final_sense):
            if direction == True:
                self.counting_left_right[3] = self.counting_left_right[3]+1
                if bigger_size<=self.faixa_a_pixels:
                    print('faixa A')
                    val_add = [0,0,0,0]
                    self.axis_left_right[0] = self.axis_left_right[0] + 1
                    val_add[0] = val_add[0]+1
                    if intermediate_sense == True:
                        for a in axis_intermediate:
                            if a>2:
                                self.axis_left_right[3] = self.axis_left_right[3] + 1
                                val_add[3] = val_add[3]+1
                            else:
                                self.axis_left_right[a+1] = self.axis_left_right[a+1] + 1
                                val_add[a+1] = val_add[a+1]+1
                    if final_sense == True:
                        for a in axis_final:
                            if a>2:
                                self.axis_left_right[3] = self.axis_left_right[3] + 1
                                val_add[3] = val_add[3]+1
                            else:
                                self.axis_left_right[a+1] = self.axis_left_right[a+1] + 1
                                val_add[a+1] = val_add[a+1]+1
                    elif final_sense ==  False:
                        self.axis_left_right[2] = self.axis_left_right[2] + 1
                        val_add[2] = val_add[2]+1
                    self.seq_objs_count_left_right.append([7,val_add,True])
                if bigger_size>self.faixa_a_pixels and bigger_size<=self.faixa_b_pixels:
                    print('faixa B')
                    val_add = [0,0,0,0]
                    self.axis_left_right[0] = self.axis_left_right[0] + 1
                    val_add[0] = val_add[0]+1
                    if intermediate_sense == True:
                        for a in axis_intermediate:
                            if a>2:
                                self.axis_left_right[3] = self.axis_left_right[3] + 1
                                val_add[3] = val_add[3]+1
                            else:
                                self.axis_left_right[a+1] = self.axis_left_right[a+1] + 1
                                val_add[a+1] = val_add[a+1]+1
                    elif intermediate_sense == False:
                        self.axis_left_right[2] = self.axis_left_right[2] + 1
                        val_add[2] = val_add[2]+1
                    if final_sense == True:
                        for a in axis_final:
                            if a>2:
                                self.axis_left_right[3] = self.axis_left_right[3] + 1
                                val_add[3] = val_add[3]+1
                            else:
                                self.axis_left_right[a+1] = self.axis_left_right[a+1] + 1
                                val_add[a+1] = val_add[a+1]+1
                    elif final_sense ==  False:
                        self.axis_left_right[3] = self.axis_left_right[3] + 1
                        val_add[3] = val_add[3]+1
                    self.seq_objs_count_left_right.append([7,val_add,True])
                if bigger_size>self.faixa_b_pixels:
                    print('faixa C')

                    val_add = [0,0,0,0]
                    self.axis_left_right[0] = self.axis_left_right[0] + 1
                    val_add[0] = val_add[0]+1
                    if intermediate_sense == True:
                        for a in axis_intermediate:
                            if a>2:
                                self.axis_left_right[3] = self.axis_left_right[3] + 1
                                val_add[3] = val_add[3]+1
                            else:
                                self.axis_left_right[a+1] = self.axis_left_right[a+1] + 1
                                val_add[a+1] = val_add[a+1]+1
                    elif intermediate_sense == False:
                        self.axis_left_right[2] = self.axis_left_right[2] + 2
                        val_add[2] = val_add[2]+2
                    if final_sense == True:
                        for a in axis_final:
                            if a>2:
                                self.axis_left_right[3] = self.axis_left_right[3] + 1
                                val_add[3] = val_add[3]+1
                            else:
                                self.axis_left_right[a+1] = self.axis_left_right[a+1] + 1
                                val_add[a+1] = val_add[a+1]+1
                    elif final_sense ==  False:
                        self.axis_left_right[2] = self.axis_left_right[2] + 2
                        val_add[2] = val_add[2]+2

                    self.seq_objs_count_left_right.append([7,val_add,True])
                    self.bitrem_frame_alert_left_right = True
            elif direction == False:
                self.counting_right_left[3] = self.counting_right_left[3]+1
                if bigger_size<=self.faixa_a_pixels:
                    print('faixa A')
                    val_add = [0,0,0,0]
                    self.axis_right_left[0] = self.axis_right_left[0] + 1
                    val_add[0] = val_add[0]+1
                    if intermediate_sense == True:
                        for a in axis_intermediate:
                            if a>2:
                                self.axis_right_left[3] = self.axis_right_left[3] + 1
                                val_add[3] = val_add[3]+1
                            else:
                                self.axis_right_left[a+1] = self.axis_right_left[a+1] + 1
                                val_add[a+1] = val_add[a+1]+1
                    if final_sense == True:
                        for a in axis_final:
                            if a>2:
                                self.axis_right_left[3] = self.axis_right_left[3] + 1
                                val_add[3] = val_add[3]+1
                            else:
                                self.axis_right_left[a+1] = self.axis_right_left[a+1] + 1
                                val_add[a+1] = val_add[a+1]+1
                    elif final_sense ==  False:
                        self.axis_right_left[2] = self.axis_right_left[2] + 1
                        val_add[2] = val_add[2]+1

                    self.seq_objs_count_right_left.append([7,val_add,False])
                if bigger_size>self.faixa_a_pixels and bigger_size<=self.faixa_b_pixels:
                    print('faixa B')
                    val_add = [0,0,0,0]                
                    self.axis_right_left[0] = self.axis_right_left[0] + 1
                    val_add[0] = val_add[0]+1
                    if intermediate_sense == True:
                        for a in axis_intermediate:
                            if a>2:
                                self.axis_right_left[3] = self.axis_right_left[3] + 1
                                val_add[3] = val_add[3]+1
                            else:
                                self.axis_right_left[a+1] = self.axis_right_left[a+1] + 1
                                val_add[a+1] = val_add[a+1]+1
                    elif intermediate_sense == False:
                        self.axis_right_left[2] = self.axis_right_left[2] + 1
                        val_add[2] = val_add[2]+1
                    if final_sense == True:
                        for a in axis_final:
                            if a>2:
                                self.axis_right_left[3] = self.axis_right_left[3] + 1
                                val_add[3] = val_add[3]+1
                            else:
                                self.axis_right_left[a+1] = self.axis_right_left[a+1] + 1
                                val_add[a+1] = val_add[a+1]+1
                    elif final_sense ==  False:
                        self.axis_right_left[3] = self.axis_right_left[3] + 1
                        val_add[3] = val_add[3]+1
                    self.seq_objs_count_right_left.append([7,val_add,False])
                if bigger_size>self.faixa_b_pixels:
                    print('faixa C')
                    val_add = [0,0,0,0]
                    self.axis_right_left[0] = self.axis_right_left[0] + 1
                    val_add[0] = val_add[0]+1
                    if intermediate_sense == True:
                        for a in axis_intermediate:
                            if a>2:
                                self.axis_right_left[3] = self.axis_right_left[3] + 1
                                val_add[3] = val_add[3]+1
                            else:
                                self.axis_right_left[a+1] = self.axis_right_left[a+1] + 1
                                val_add[a+1] = val_add[a+1]+1
                    elif intermediate_sense == False:
                        self.axis_right_left[2] = self.axis_right_left[2] + 2
                        val_add[2] = val_add[2]+2
                    if final_sense == True:
                        for a in axis_final:
                            if a>2:
                                self.axis_right_left[3] = self.axis_right_left[3] + 1
                                val_add[3] = val_add[3]+1
                            else:
                                self.axis_right_left[a+1] = self.axis_right_left[a+1] + 1
                                val_add[a+1] = val_add[a+1]+1
                    elif final_sense ==  False:
                        self.axis_right_left[2] = self.axis_right_left[2] + 2
                        val_add[2] = val_add[2]+2
                    self.seq_objs_count_right_left.append([7,val_add,False])
                    self.bitrem_frame_alert_right_left = True
        intermediate_sense,final_sense = makes_sense(axis_intermediate,axis_final)
        counting(intermediate_sense,final_sense)

    # Atualiza todos os rastreadores
    def update_trackers(self,frame,move_boxes):                    
        pos_objects = []
        for i, one_object in enumerate(self.objects):
            pos_before = one_object[1][0]
            success, pos_one_object = one_object[0].update(frame)
            pos_after = pos_one_object[0]
            one_object[1] = pos_one_object
            pos_objects.append(pos_one_object)

            point_box = one_object[7][0]+one_object[7][2]

            p1 = abs(one_object[1][0]-self.x1_count_line_light)
            p2 = abs(one_object[1][0]+one_object[1][2]-self.x1_count_line_light)

            if pos_before<pos_after:
                #Sentido do veículo é --->
                one_object[9] = True
            elif pos_before>pos_after:
                #Sentido do veículo é <---
                one_object[9] = False

            #print(one_object[7][0])
            x1 = 0
            x2 = 0
            if one_object[2] == 7 or one_object[2] == 5:
                x1 = self.x1_count_line_heavy
                x2 = self.x1_count_line2_heavy 
            else:
                x1 = self.x1_count_line_light
                x2 = self.x1_count_line2_light
            
            var = False
            if one_object[2] == 7:    
                for move in move_boxes:
                    if one_object[1][0]>move[0] and one_object[1][1]>move[1] and one_object[1][0]+one_object[1][2]<move[0]+move[2] and one_object[1][1]+one_object[1][3]<move[1]+move[3]:
                        if move[2]>500:
                            var = True

            if one_object[9] == True and one_object[7][0]>x1:
                #print('removed --->')
                #car
                if one_object[2] == 2 and one_object[3] == False:
                    self.counting_left_right[0] = self.counting_left_right[0]+1
                    one_object[3] = True
                #moto
                elif one_object[2] == 3 and one_object[3] == False:
                    self.counting_left_right[1] = self.counting_left_right[1]+1
                    one_object[3] = True
                #bus
                elif one_object[2] == 5 and one_object[3] == False:
                    self.counting_left_right[2] = self.counting_left_right[2]+1
                    if max(one_object[5])>2:
                        self.axis_left_right[0] = self.axis_left_right[0] + 1
                        self.axis_left_right[1] = self.axis_left_right[1] + 2
                    else:
                        self.axis_left_right[0] = self.axis_left_right[0] + 1
                        self.axis_left_right[1] = self.axis_left_right[1] + 1
                    one_object[3] = True
                #truck
                elif one_object[2] == 7 and one_object[3] == False and var == False:
                    
                    one_object[3] = True

                    if (one_object[11] >=2 and len(self.seq_objs_count_left_right)>0) or one_object[12] == True:
                        print('BITREM')
                        last_classId = self.seq_objs_count_left_right[-1][0]
                        if last_classId == 7:
                            last_esrs = self.seq_objs_count_left_right[-1][1][0]
                            last_esrd = self.seq_objs_count_left_right[-1][1][1]
                            last_etd = self.seq_objs_count_left_right[-1][1][2]
                            last_ett = self.seq_objs_count_left_right[-1][1][3]

                            self.axis_left_right[0] = self.axis_left_right[0] - last_esrs
                            self.axis_left_right[1] = self.axis_left_right[1] - last_esrd
                            self.axis_left_right[2] = self.axis_left_right[2] - last_etd
                            self.axis_left_right[3] = self.axis_left_right[3] - last_ett
                            
                            self.axis_left_right[0] = self.axis_left_right[0] + 1
                            self.axis_left_right[2] = self.axis_left_right[2] + 4
                    else:
                    
                        bigger_size = max(one_object[10])
                        interm,final = self.wheel_ident_avaliator(one_object[6])
                        self.truck_size(one_object[9],bigger_size,interm,final)

                print(one_object[6])
                obj = self.objects.pop(i) 

            if one_object[9] == False and one_object[7][0]+one_object[7][2]<x2:
                print('Removed <---')    
                if one_object[2] == 2 and one_object[3] == False:
                    self.counting_right_left[0] = self.counting_right_left[0]+1
                    one_object[3] = True
                elif one_object[2] == 3 and one_object[3] == False:
                    self.counting_right_left[1] = self.counting_right_left[1]+1
                    one_object[3] = True
                elif one_object[2] == 5 and one_object[3] == False:
                    self.counting_right_left[2] = self.counting_right_left[2]+1
                    if max(one_object[5])>2:
                        self.axis_right_left[0] = self.axis_right_left[0] + 1
                        self.axis_right_left[1] = self.axis_right_left[1] + 2
                    else:
                        self.axis_right_left[0] = self.axis_right_left[0] + 1
                        self.axis_right_left[1] = self.axis_right_left[1] + 1
                    one_object[3] = True
                elif one_object[2] == 7 and one_object[3] == False and var == False:
                    
                    one_object[3] = True
                    
                    if (one_object[11] >=2 and len(self.seq_objs_count_right_left)>0) or one_object[12] == True:
                        print('BITREM')
                        last_classId = self.seq_objs_count_right_left[-1][0]
                        if last_classId == 7:
                            last_esrs = self.seq_objs_count_right_left[-1][1][0]
                            last_esrd = self.seq_objs_count_right_left[-1][1][1]
                            last_etd = self.seq_objs_count_right_left[-1][1][2]
                            last_ett = self.seq_objs_count_right_left[-1][1][3]

                            self.axis_right_left[0] = self.axis_right_left[0] - last_esrs
                            self.axis_right_left[1] = self.axis_right_left[1] - last_esrd
                            self.axis_right_left[2] = self.axis_right_left[2] - last_etd
                            self.axis_right_left[3] = self.axis_right_left[3] - last_ett
                            
                            self.axis_right_left[0] = self.axis_right_left[0] + 1
                            self.axis_right_left[2] = self.axis_right_left[2] + 4
                    
                    else:
                        print('RODANDO')
                        print(one_object)
                        bigger_size = max(one_object[10])
                        interm,final = self.wheel_ident_avaliator(one_object[6])
                        self.truck_size(one_object[9],bigger_size,interm,final)
                    
                    
                print(one_object[6])
                obj = self.objects.pop(i)  
            self.update_indexes()
        return pos_objects,self.objects  

    
            
    def run(self):
        
        while(self.cap.isOpened()):

            init_time = time()
            # Read a new frame
            ok, frame = self.cap.read()
            if not ok:
                break
            
            is_movement = False
            move,fgmask,countors = find_movement(frame.copy(),self.subtractor)
            
            for m in move:
                if (m[0]>self.x1_count_line_heavy and m[0]<self.x1_count_line2_heavy) or (m[0]+m[2]>self.x1_count_line_heavy and m[0]+m[2]<self.x1_count_line2_heavy) or (m[0]<self.x1_count_line_heavy and m[0]+m[2]>self.x1_count_line2_heavy):
                        is_movement = True
                        break

            if self.bitrem_frame_alert_left_right == True:
                self.bitrem_frame_count_left_right = self.bitrem_frame_count_left_right+1
                if self.bitrem_frame_count_left_right>15:
                    self.bitrem_frame_alert_left_right = False
                    self.bitrem_frame_count_left_right = 0
            if self.bitrem_frame_alert_right_left == True:
                self.bitrem_frame_count_right_left = self.bitrem_frame_count_right_left+1
                if self.bitrem_frame_count_right_left>15:
                    self.bitrem_frame_alert_right_left = False
                    self.bitrem_frame_count_right_left = 0            
                         
            self.num_frame = self.num_frame+1
            
            if is_movement ==True:

                # pega a localização dos rastreadores no frame
                tracker_boxes_updated, objects_updated = self.update_trackers(frame,move)

                # Criar a blob a ser processada pela rede neural
                blob = cv2.dnn.blobFromImage(frame, 1/255, (self.inpWidth, self.inpHeight), [0,0,0], 1, crop=False)

                # Passa a blob para a rede neural
                self.net.setInput(blob)

                # Receve o resultado da previsão da rede neural
                outs = self.net.forward(getOutputsNames(self.net))

                # Remove bounding boxes com baixa confiança
                boxes,classIds,ids_centers = postprocess(frame, outs) 
                objects_set = [boxes,classIds]
                fr,axis_centers,final_type = self.axis_counting.identify_axis(frame,objects_set,objects_updated,self.sess)

                self.tracker_config(boxes,objects_updated,classIds,frame,axis_centers,move)    
                self.check_trackers(boxes)
                
                for i,ax in enumerate(self.axis_left_right):
                    if ax < 0:
                        self.axis_left_right[i] = 0
                for i,ax in enumerate(self.axis_right_left):
                    if ax < 0:
                        self.axis_right_left[i] = 0

                # Desenha rastreadores
                for i, newbox in enumerate(tracker_boxes_updated):
                    p1 = (int(newbox[0]), int(newbox[1]))
                    p2 = (int(newbox[0]+newbox[2]), int(newbox[1]+newbox[3])) 
                    cv2.rectangle(frame, p1, p2, (255,0,0), 2, 1)

            self.update_indexes()
            
            img = cv2.rectangle(frame,(int(width/6),height-115),(int(5*width/6),height-10),(255,255,255),-1)

            ######## Display car counting left to right
            cv2.putText(frame, 'Faixa 1 --> ', (int(width/6)+10,height-100), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1);

            cv2.putText(frame, 'Car: ' + str(self.counting_left_right[0]), (int(width/6)+10,height-80), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1);

            cv2.putText(frame, 'Bus: ' + str(self.counting_left_right[2]), (int(width/6)+10,height-60), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1);

            cv2.putText(frame, 'Truck: ' + str(self.counting_left_right[3]), (int(width/6)+10,height-40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1);

            cv2.putText(frame, 'ESRS: ' + str(self.axis_left_right[0]), (int(width/3)+30,height-80), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1);

            cv2.putText(frame, 'ESRD: ' + str(self.axis_left_right[1]), (int(width/3)+30,height-60), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1);

            cv2.putText(frame, 'ETD: ' + str(self.axis_left_right[2]), (int(width/3)+30,height-40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1);

            cv2.putText(frame, 'ETT: ' + str(self.axis_left_right[3]), (int(width/3)+30,height-20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1);

            ######## Display car counting right to left
            cv2.putText(frame, 'Faixa 2 <-- ', (int(width/2)+20,height-100), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1);

            cv2.putText(frame, 'Car: ' + str(self.counting_right_left[0]), (int(width/2)+20,height-80), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1);

            cv2.putText(frame, 'Bus: ' + str(self.counting_right_left[2]), (int(width/2)+20,height-60), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1);

            cv2.putText(frame, 'Truck: ' + str(self.counting_right_left[3]), (int(width/2)+20,height-40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1);

            cv2.putText(frame, 'ESRS: ' + str(self.axis_right_left[0]), (int(width/2)+140,height-80), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1);

            cv2.putText(frame, 'ESRD: ' + str(self.axis_right_left[1]), (int(width/2)+140,height-60), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1);

            cv2.putText(frame, 'ETD: ' + str(self.axis_right_left[2]), (int(width/2)+140,height-40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1);

            cv2.putText(frame, 'ETT: ' + str(self.axis_right_left[3]), (int(width/2)+140,height-20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1);


            # Display car counting
            video_percent = round(((self.start_frame+self.num_frame)/self.total_num_frames)*100,2)
            cv2.putText(frame, 'Frame: ' + str(self.start_frame+self.num_frame) + '/' + str(self.total_num_frames) + '   ' + str(video_percent) + '%', (int(width)-300,30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0,0), 1);

            img = cv2.line(frame,(self.x1_count_line_light,self.y1_count_line_light),(self.x2_count_line_light,self.y2_count_line_light),(0,165,255),3)

            img = cv2.line(frame,(self.x1_count_line2_light,self.y1_count_line2_light),(self.x2_count_line2_light,self.y2_count_line2_light),(0,165,255),3)

            img = cv2.line(frame,(self.x1_count_line_heavy,self.y1_count_line_heavy),(self.x2_count_line_heavy,self.y2_count_line_heavy),(255,100,0),3)

            img = cv2.line(frame,(self.x1_count_line2_heavy,self.y1_count_line2_heavy),(self.x2_count_line2_heavy,self.y2_count_line2_heavy),(255,100,0),3)

            img = cv2.line(frame,(int(width/2),self.y1_count_line_light),(int(width/2),self.y2_count_line_light),(0,0,255),3)

            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                        
            gray = cv2.GaussianBlur(gray, (11, 11), 0)

    
            draw_movement(frame,move)

            # Escreve o frame no video do processamento que está sendo gravado
            self.out.write(frame)
            # Mostra o resultado de cada frame
            cv2.imshow('frame',frame)

            if self.start_frame + self.num_frame == self.total_num_frames-10:
                print(time()-init_time)
                break
            if cv2.waitKey(1) & 0xFF == ord('q'):
                print(time()-init_time)
                break     
                
        self.cap.release()
        self.out.release()
        cv2.destroyAllWindows()
        print(self.axis_left_right)
        print(self.axis_right_left)
        print(self.counting_left_right)
        print(self.counting_right_left)

run_video()

26.181818181818183
[]
[[0, 0.8371458053588867]]
[[0, 0.8283691201158749]]
[]
[[0, 0.2954233980178833]]
[]
[]
[[0, 0.6377124637365341], [0, 0.9539299309253693]]
[]
[[0, 0.9188986658596721]]
[[0, 0.6018809498016244], [0, 0.8938050217294166]]
[[0, 0.2851003011067708]]
[[0, 0.28159312748603893], [1, 0.8891772910053595]]
[[0, 0.916301479408457]]
[[0, 0.8978036482703877]]
[[0, 0.8019677148738378]]
[[0, 0.2609459586765455], [0, 0.8598850416100543]]
[[0, 0.8402810548135623]]
[[0, 0.8127398847722683]]
[[0, 0.8138396147709743]]
[[0, 0.7964181110357783]]
[[0, 0.7964181110357783]]
[[0, 0.7670157851686903]]
[[0, 0.7670157851686903]]
[[0, 0.7621577692488892]]
[[0, 0.7621577692488892]]
[[0, 0.7118815827293518]]
[]
Removed <---
[[], [[0, 0.8371458053588867]], [[0, 0.8283691201158749]], [], [[0, 0.2954233980178833]], [], [], [[0, 0.6377124637365341], [0, 0.9539299309253693]], [], [[0, 0.9188986658596721]], [[0, 0.6018809498016244], [0, 0.8938050217294166]], [[0, 0.2851003011067708]], [[0, 0.28159312748

[[0, 0.17300405381601067], [0, 0.17400405381601067], [3, 0.8163121863256527]]
